# Image Forensics Analysis with Veridex

This notebook demonstrates how to use Veridex to detect AI-generated images.

We will cover:
1. **Frequency Analysis**: Detecting artifacts in the frequency domain.
2. **DIRE (Diffusion Reconstruction Error)**: Using reverse diffusion to identify synthetic content.

In [ ]:
# Install dependencies
!pip install veridex[image]

In [ ]:
from veridex.image import FrequencySignal, DIRESignal
import requests
from PIL import Image
from io import BytesIO
import os

## 1. Prepare Data

We will download a sample real image and (if available) use a known AI image. For this demo, we'll just download a standard test image.

In [ ]:
def download_image(url, path):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img.save(path)
    return path

# Sample real image (Lenna or standard test image)
real_img_url = "https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png"
real_img_path = "sample_real.png"

try:
    download_image(real_img_url, real_img_path)
    print("Downloaded sample image.")
except Exception as e:
    print(f"Failed to download: {e}")
    # Create dummy image if download fails
    Image.new('RGB', (256, 256), color='red').save(real_img_path)

## 2. Frequency Signal

Analyzes the Fast Fourier Transform (FFT) of the image.

In [ ]:
freq_detector = FrequencySignal()

res_freq = freq_detector.detect(real_img_path)

print(f"Frequency Score (Real Image): {res_freq.score:.4f}")
print(f"Confidence: {res_freq.confidence:.4f}")

## 3. DIRE Signal

This checks how easily the image can be reconstructed by Stable Diffusion. 

**Note:** This requires a GPU to run efficiently. It will be very slow on CPU.

In [ ]:
try:
    # You can specify device='cpu' if you don't have a GPU, but it will be slow.
    dire_detector = DIRESignal(device='cpu') 
    
    print("Running DIRE detection (this may take a while)...\n")
    res_dire = dire_detector.detect(real_img_path)
    
    print(f"DIRE Score: {res_dire.score:.4f}")
    print(f"Error Map Mean: {res_dire.metadata.get('reconstruction_error', 'N/A')}")

except ImportError:
    print("DIRE dependencies not installed. Install via `pip install veridex[image]`")
except Exception as e:
    print(f"DIRE execution failed: {e}")